### Loading the libraries

In [1]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.6.0

In [2]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.3.2

### Import the classes

In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators


In [4]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


### Instantiate Spark Instance

In [5]:
val conf = new SparkConf().setMaster("local[*]").setAppName("Airbnb analysis")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@4e32ff0d

### Schema and Feature Creation

In [20]:
//City,City Group,Type,revenue
case class SimpleRegression2 (
  city: String,
  cityGroup: String,
  rType: String,
  revenue: Double
)

defined class SimpleRegression2


In [21]:
val city = FeatureBuilder.Text[SimpleRegression2].extract(_.city.toText).asPredictor
val cityGroup = FeatureBuilder.Text[SimpleRegression2].extract(_.cityGroup.toText).asPredictor
val rType = FeatureBuilder.Text[SimpleRegression2].extract(_.rType.toText).asPredictor
val revenue = FeatureBuilder.RealNN[SimpleRegression2].extract(_.revenue.toRealNN).asResponse

Feature(name = revenue, uid = RealNN_00000000001e, isResponse = true, originStage = FeatureGeneratorStage_00000000001e, parents = [], distributions = [])

#### Load the data

In [42]:
import spark.implicits._

val trainFilePath = "./data/restaurant_train_13700_rows.csv"
val trainDataReader = DataReaders.Simple.csvCase[SimpleRegression2](
    path = Option(trainFilePath)
)

org.apache.spark.sql.SparkSession$implicits$@4f331fc7

In [49]:
// check that path exists
scala.reflect.io.File(trainFilePath).exists

true

In [50]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val features = Seq(city, cityGroup, rType).transmogrify()
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_00000000006f

#### Model Selector
The ModelSelector is an Estimator that uses data to find the best model. 

In [51]:
//import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpGBTRegressor, OpRandomForestRegressor,OpLinearRegression}
import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpLinearRegression}
import com.salesforce.op.stages.impl.regression.RegressionModelSelector

val cutter = DataCutter(reserveTestFraction = 0.2, seed = randomSeed)

val prediction = RegressionModelSelector
      .withCrossValidation(
        dataSplitter = Some(splitter), seed = randomSeed,
        //modelTypesToUse = Seq(OpGBTRegressor, OpRandomForestRegressor)
        //modelTypesToUse = Seq(OpLinearRegression, 
        //                      OpRandomForestRegressor)
        modelTypesToUse = Seq(OpLinearRegression)
        
 
).setInput(revenue,features).getOutput()

Feature(name = city-cityGroup-rType-revenue_3-stagesApplied_Prediction_000000000079, uid = Prediction_000000000079, isResponse = true, originStage = ModelSelector_000000000079, parents = [RealNN_00000000001e,OPVector_00000000006e], distributions = [])

#### Evaluators and Workflow
Factory that performs the evaluation of metrics for regression. The metrics returned are rmse, mse, r2 and mae.
* Mean Squared Error (MSE)	
* Root Mean Squared Error (RMSE)	
* Mean Absolute Error (MAE)	
* Coefficient of Determination 

OpWorkflows create and transform the raw data needed to compute Features fed into them. In addition they optimize the application of Stages needed to create the final Features ensuring optimal computations within the full pipeline DAG. OpWorkflows can be fit to a given dataset using the .train() method. This produces an OpWorkflowModel which can then be saved to disk and applied to another dataset.

In [52]:
val evaluator = Evaluators.Regression().setLabelCol(revenue).
      setPredictionCol(prediction)

OpRegressionEvaluator_00000000007a

In [53]:
val workflow = new OpWorkflow().setResultFeatures(prediction, revenue).setReader(trainDataReader)
val workflowModel = workflow.train()

com.salesforce.op.OpWorkflowModel@408a7105

#### Score and evaluate

In [54]:
val dfScoreAndEvaluate = workflowModel.scoreAndEvaluate(evaluator)
val dfScore = dfScoreAndEvaluate._1.withColumnRenamed("city-cityGroup-rType-revenue_3-stagesApplied_Prediction_000000000049","predicted_revenue")
val dfEvaluate = dfScoreAndEvaluate._2
println("Evaluate:\n" + dfEvaluate.toString())

dfScore.show(false)

Evaluate:
{
  "RootMeanSquaredError" : 2245000.1294779773,
  "MeanSquaredError" : 5.040025581356136E12,
  "R2" : 0.2178718317114826,
  "MeanAbsoluteError" : 1378234.1647755317
}
+--------------------+---------+--------------------------------------------------------------------+
|key                 |revenue  |city-cityGroup-rType-revenue_3-stagesApplied_Prediction_000000000079|
+--------------------+---------+--------------------------------------------------------------------+
|-1233186009812836990|3452382.0|[prediction -> 2225088.2256419715]                                  |
|-5422781442898149234|3248660.0|[prediction -> 2348658.5030552177]                                  |
|2078917795437629936 |3376145.0|[prediction -> 3376145.12113678]                                    |
|3806169214982705301 |3600467.0|[prediction -> 6095018.414235921]                                   |
|-3855255903276174641|3273041.0|[prediction -> 2631004.8342450066]                                  |
|-3490

null